# PAndas preprocessing

In [ ]:
import pandas as pd

In [ ]:
df =  pd.read_html('https://en.wikipedia.org/wiki/List_of_chapters_in_the_Quran')
df = df[0]

In [ ]:
df.columns

In [ ]:
df2 = df[['Anglicized title(s)', 'Number of verses (Number of Rukūʿs)']]

In [ ]:
df2

In [ ]:
df2['Number of verses (Number of Rukūʿs)'] = df2['Number of verses (Number of Rukūʿs)'].str.split(' ', expand=True)[0]

In [ ]:
df2

In [ ]:
df2.to_csv('surah_aayah_count.csv', index=False)

# Scraping tests

In [ ]:
import pandas as pd
import numpy as np
import requests
import os

In [ ]:
surah_df = pd.read_csv('surah_aayah_count.csv').dropna()

def generate_dateset(imam):
    output_dir = 'data'
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    output_dir = os.path.join(output_dir, imam)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    for surah_index in range(0, 113):
        for aayah_number in range(1, int(surah_df.iloc[surah_index]['Number of verses (Number of Rukūʿs)'] + 1)):
            file_name = f'{str(surah_index+1).zfill(3)}{str(aayah_number).zfill(3)}'
            url = f"https://everyayah.com/data/{imam}/{file_name}.mp3"

            out_file_path = os.path.join(output_dir, f'{surah_index+1}_{aayah_number}.mp3')
            if os.path.exists(out_file_path):
                continue
            doc = requests.get(url)

            if doc.status_code == 200:
                print(f'{surah_index+1}_{aayah_number}.mp3')
                with open(out_file_path, 'wb') as f:
                    f.write(doc.content)
            else:
                raise Exception(f'Error: {doc.status_code} for {url}')

In [ ]:
imam_df = pd.read_csv('audiocats.csv')
df_chunks = np.array_split(imam_df, 10)

In [ ]:
df_chunks

In [ ]:
def generate_dataset_parallel(df):
    for imam in df['imam'].unique():
        generate_dateset(imam)

In [ ]:
from multiprocessing import Pool

with Pool(8) as p:
    print(p.map(generate_dataset_parallel, df_chunks))

# Regex experiment

In [10]:
import pandas as pd
import re

In [16]:
df = pd.read_excel('dataset 121.xlsx', sheet_name='Sheet4', header=[None])

In [17]:
df

,0
0,بِسْمِ ٱللَّهِ ٱلرَّحْمَـٰنِ ٱلرَّحِيمِ
1,ٱلْحَمْدُ لِلَّهِ رَبِّ ٱلْعَـٰلَمِينَ
2,ٱلرَّحْمَـٰنِ ٱلرَّحِيمِ
3,مَـٰلِكِ يَوْمِ ٱلدِّينِ
4,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
...,...
6263,# derived from or containing substantial po...
6264,#
6265,# Please check updates at: http://tanzil.net/...
6266,#


In [49]:
aayah = '2|3|ٱلَّذِينَ يُؤْمِنُونَ بِٱلْغَيْبِ وَيُقِيمُونَ ٱلصَّلَوٰةَ وَمِمَّا رَزَقْنَـٰهُمْ يُنفِقُونَ'

In [50]:
aayah.replace(r'[0-9]', '')   

'2|3|ٱلَّذِينَ يُؤْمِنُونَ بِٱلْغَيْبِ وَيُقِيمُونَ ٱلصَّلَوٰةَ وَمِمَّا رَزَقْنَـٰهُمْ يُنفِقُونَ'

In [51]:
re.sub(r'[a-z]|[A-Z]|[0-9]|\|', '', aayah)

'ٱلَّذِينَ يُؤْمِنُونَ بِٱلْغَيْبِ وَيُقِيمُونَ ٱلصَّلَوٰةَ وَمِمَّا رَزَقْنَـٰهُمْ يُنفِقُونَ'

In [26]:
df = df.dropna()

In [66]:
df2 = df[0].apply(lambda x: re.sub(r'[a-z]|[A-Z]|[0-9]|\|', '', x))

In [67]:
df2.to_excel('cleaned_dataset.xlsx', index=False)

In [53]:
df2.head(20)

0               بِسْمِ ٱللَّهِ ٱلرَّحْمَـٰنِ ٱلرَّحِيمِ
1                ٱلْحَمْدُ لِلَّهِ رَبِّ ٱلْعَـٰلَمِينَ
2                              ٱلرَّحْمَـٰنِ ٱلرَّحِيمِ
3                              مَـٰلِكِ يَوْمِ ٱلدِّينِ
4              إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
5                    ٱهْدِنَا ٱلصِّرَٰطَ ٱلْمُسْتَقِيمَ
6     صِرَٰطَ ٱلَّذِينَ أَنْعَمْتَ عَلَيْهِمْ غَيْرِ...
8               بِسْمِ ٱللَّهِ ٱلرَّحْمَـٰنِ ٱلرَّحِيمِ
9                                                 الٓمٓ
10    ذَٰلِكَ ٱلْكِتَـٰبُ لَا رَيْبَ فِيهِ هُدًى لِّ...
11    ٱلَّذِينَ يُؤْمِنُونَ بِٱلْغَيْبِ وَيُقِيمُونَ...
12    وَٱلَّذِينَ يُؤْمِنُونَ بِمَآ أُنزِلَ إِلَيْك...
13    أُو۟لَـٰٓئِكَ عَلَىٰ هُدًى مِّن رَّبِّهِمْ وَأ...
14    إِنَّ ٱلَّذِينَ كَفَرُوا۟ سَوَآءٌ عَلَيْهِمْ ...
15    خَتَمَ ٱللَّهُ عَلَىٰ قُلُوبِهِمْ وَعَلَىٰ سَم...
16    وَمِنَ ٱلنَّاسِ مَن يَقُولُ ءَامَنَّا بِٱللَّه...
17    يُخَـٰدِعُونَ ٱللَّهَ وَٱلَّذِينَ ءَامَنُوا۟ و...
18    فِى قُلُوبِهِم مَّرَضٌ فَزَادَهُمُ ٱللَّهُ

In [59]:
df[0].str.replace(r'([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+', '').head(20)

0               بِسْمِ ٱللَّهِ ٱلرَّحْمَـٰنِ ٱلرَّحِيمِ
1                ٱلْحَمْدُ لِلَّهِ رَبِّ ٱلْعَـٰلَمِينَ
2                              ٱلرَّحْمَـٰنِ ٱلرَّحِيمِ
3                              مَـٰلِكِ يَوْمِ ٱلدِّينِ
4              إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
5                    ٱهْدِنَا ٱلصِّرَٰطَ ٱلْمُسْتَقِيمَ
6     صِرَٰطَ ٱلَّذِينَ أَنْعَمْتَ عَلَيْهِمْ غَيْرِ...
8               بِسْمِ ٱللَّهِ ٱلرَّحْمَـٰنِ ٱلرَّحِيمِ
9                                                 الٓمٓ
10    ذَٰلِكَ ٱلْكِتَـٰبُ لَا رَيْبَ فِيهِ هُدًى لِّ...
11    2|3|ٱلَّذِينَ يُؤْمِنُونَ بِٱلْغَيْبِ وَيُقِيم...
12    2|4|وَٱلَّذِينَ يُؤْمِنُونَ بِمَآ أُنزِلَ إِل...
13    2|5|أُو۟لَـٰٓئِكَ عَلَىٰ هُدًى مِّن رَّبِّهِمْ...
14    2|6|إِنَّ ٱلَّذِينَ كَفَرُوا۟ سَوَآءٌ عَلَيْه...
15    2|7|خَتَمَ ٱللَّهُ عَلَىٰ قُلُوبِهِمْ وَعَلَىٰ...
16    2|8|وَمِنَ ٱلنَّاسِ مَن يَقُولُ ءَامَنَّا بِٱل...
17    2|9|يُخَـٰدِعُونَ ٱللَّهَ وَٱلَّذِينَ ءَامَنُو...
18    2|10|فِى قُلُوبِهِم مَّرَضٌ فَزَادَهُمُ ٱل

In [61]:
re.sub(r'([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+', '', aayah)

'ٱلذين يؤمنون بٱلغيب ويقيمون ٱلصلوة ومما رزقنهم ينفقون'

# Aayah separation

In [68]:
import pandas as pd

In [73]:
df = pd.read_csv('Arabic-Original.csv', delimiter='|', header=None)

In [75]:
df.columns = ['surah_number', 'aayah_number', 'aayah']

In [76]:
df

,surah_number,aayah_number,aayah
0,1,1,بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ
1,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
2,1,3,الرَّحْمَٰنِ الرَّحِيمِ
3,1,4,مَالِكِ يَوْمِ الدِّينِ
4,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
...,...,...,...
6231,114,2,مَلِكِ النَّاسِ
6232,114,3,إِلَٰهِ النَّاسِ
6233,114,4,مِنْ شَرِّ الْوَسْوَاسِ الْخَنَّاسِ
6234,114,5,الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ


In [77]:
df[df['aayah'] == 'بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ']

,surah_number,aayah_number,aayah
0,1,1,بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ
